In [4]:
import pandas as pd
from datetime import datetime
from calendar import monthrange
import seaborn as sns

In [5]:
missing_values = ["-"]
df = pd.read_csv("History of Ice on Lake Mendota.csv", na_values=missing_values)
df.head()

FileNotFoundError: File b'History of Ice on Lake Mendota.csv' does not exist

In [ ]:
df.isnull().sum()

In [ ]:
df['DAYS'].fillna(0, inplace=True)
df

In [ ]:
def transformYear(row):
    closedMonth = datetime.strptime(row['CLOSED'], '%d %b').month
    openedMonth = datetime.strptime(row['OPENED'], '%d %b').month
    if closedMonth in [11, 12] and openedMonth in [11, 12]:
        row['FULL_CLOSED'] = str(row['START YEAR']) + ' ' + row['CLOSED']
        row['FULL_OPENED'] = str(row['START YEAR']) + ' ' + row['OPENED']
    elif closedMonth not in [11, 12]:
        row['FULL_CLOSED'] = str(row['END YEAR']) + ' ' + row['CLOSED']
        row['FULL_OPENED'] = str(row['END YEAR']) + ' ' + row['OPENED']
    else:
        row['FULL_CLOSED'] = str(row['START YEAR']) + ' ' + row['CLOSED']
        row['FULL_OPENED'] = str(row['END YEAR']) + ' ' + row['OPENED']
    
    row['FULL_CLOSED'] = datetime.strptime(row['FULL_CLOSED'], '%Y %d %b').date()
    row['FULL_OPENED'] = datetime.strptime(row['FULL_OPENED'], '%Y %d %b').date()
    return row

In [ ]:
df = df.apply(lambda x: transformYear(x), axis=1)
df

In [ ]:
from collections import OrderedDict
months = OrderedDict([(11,'NOV'), (12,'DEC'), (1,'JAN'), (2,'FEB'), (3,'MAR'), (4,'APR'), (5,'MAY')])
months

In [ ]:
def getDaysPerMonth(test_df, average=False):
    START = False
    full_closed = test_df['FULL_CLOSED']
    full_opened = test_df['FULL_OPENED']
    if (full_closed.year == full_opened.year) and (full_closed.month == full_opened.month):
        for month in months:
            if month == full_closed.month:
                test_df[months[month]] = (full_opened.day - full_closed.day + 1)/monthrange(full_closed.year, month)[1]
            else:
                test_df[months[month]] = 0
    else:
        for month in months:
            year = full_closed.year if month in [11, 12] else full_opened.year
            if month == full_closed.month:
                START = True
                daysPerMonth = monthrange(full_closed.year, month)[1] - full_closed.day + 1
            elif month == full_opened.month:
                START = False
                daysPerMonth = full_opened.day
            else:
                if(START):
                    daysPerMonth = monthrange(year, month)[1]
                else:
                    daysPerMonth = 0
            
            test_df[months[month]] = daysPerMonth if not average else daysPerMonth/monthrange(year, month)[1]
    return test_df

In [ ]:
df = df.apply(lambda x: getDaysPerMonth(x, average=True), axis=1)
df

In [ ]:
freeze_df = df.filter(['START YEAR', 'NOV', 'DEC', 'JAN', 'FEB', 'MAR', 'APR', 'MAY'], axis=1)
freeze_df

In [ ]:
freeze_df = freeze_df.groupby(by='START YEAR').sum()
freeze_df

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,30))
ax.set_title("History of Ice on Lake Mendota", fontsize=25, fontweight='bold', pad=15)
sns.heatmap(
    freeze_df, 
    ax=ax, 
    cmap='Blues_r', 
    cbar_kws=dict(use_gridspec=False,location='bottom',pad=0.02)
)
cbar = ax.collections[0].colorbar
cbar.set_label('Ratio of closed days per month')
plt.save('history_heatmap123.jpg')
plt.show()